In [ ]:

%run includes/3-network-setup.ipynb

# Networks
Summary and detail reports on networks and subnetworks deployed across the GCP estate.

In [ ]:

# get networks
network_queries = [
    f"""
    SELECT name,
    '{project}' as project,
    mtu,
    networkFirewallPolicyEnforcementOrder as fwPolicyEnforcementOrder,
    JSON_ARRAY_LENGTH(peerings) as num_peerings,
    peerings,
    JSON_EXTRACT(routingConfig, '$.routingMode') as routingMode,
    JSON_ARRAY_LENGTH(subnetworks) as num_subnets
    FROM google.compute.networks WHERE project = '{project}'
    AND name IS NOT NULL
    """
    for project in all_projects
]
networks_df = run_stackql_queries(network_queries)
networked_projects = list(networks_df['project'].unique())

# get interconnects
interconnect_queries = [
    f"""
    SELECT *
    FROM google.compute.interconnects WHERE project = '{project}'
    AND name IS NOT NULL
    """
    for project in networked_projects
]
interconnects_df = run_stackql_queries(interconnect_queries)

# get vpns
vpn_queries = [
    f"""
    SELECT
    name,
    '{project}' as project, 
    SPLIT_PART(network, '/', 9) as network,
    SPLIT_PART(region, '/', 8) as region,
    JSON_ARRAY(JSON_EXTRACT(vpnInterfaces, '$[0].ipAddress'), JSON_EXTRACT(vpnInterfaces, '$[1].ipAddress')) as ip_addresses
    FROM google.compute.vpn_gateways WHERE project = '{project}'
    """
    for project in networked_projects
]
vpns_df = run_stackql_queries(vpn_queries)

external_vpn_gateways_queries = [
    f"""
    select 
    name
    ,description
    ,'{project}' as project
    ,JSON_ARRAY_LENGTH(interfaces) as num_interfaces
    ,JSON_ARRAY(JSON_EXTRACT(interfaces, '$[0].ipAddress'), JSON_EXTRACT(interfaces, '$[1].ipAddress'), JSON_EXTRACT(interfaces, '$[2].ipAddress'), JSON_EXTRACT(interfaces, '$[3].ipAddress')) as ip_addresses
    ,redundancyType
    ,labels 
    from google.compute.external_vpn_gateways where project = '{project}'
    and name is not null
    """
    for project in networked_projects
]
external_vpn_gateways_df = run_stackql_queries(external_vpn_gateways_queries)

# get firewalls
firewall_queries = [
    f"""
    select 
    name
    ,'{project}' as project
    ,sourceRanges
    ,destinationRanges
    ,sourceServiceAccounts
    ,sourceTags
    ,targetServiceAccounts
    ,targetTags
    ,denied
    ,direction
    ,SPLIT_PART(network, '/', 10) as network
    ,allowed
    ,priority
    ,disabled
    ,logConfig
    ,description
    from google.compute.firewalls where project = '{project}' and name is not null
    """
    for project in networked_projects
]
firewalls_df = run_stackql_queries(firewall_queries)
firewall_projects = list(firewalls_df['project'].unique())

# get firewalls with public access
fw_public_access_queries = [
    f"""
    SELECT sourceRange, * FROM
        (
        SELECT  
            name
            ,JSON_EXTRACT(sourceRanges, '$[0]') as sourceRange
            ,destinationRanges
            ,sourceServiceAccounts
            ,sourceTags
            ,targetServiceAccounts
            ,targetTags
            ,direction
            ,SPLIT_PART(network, '/', 10) as network
            ,JSON_EXTRACT(denied, '$[0].ports[0]') as denied_ports
            ,JSON_EXTRACT(denied, '$[0].IPProtocol') as denied_protocols
            ,allowed
            ,priority
            ,disabled
            ,logConfig
            ,description
        from google.compute.firewalls where project = '{project}' and name is not null
        ) as firewalls
    WHERE sourceRange = '0.0.0.0/0'
    """
    for project in firewall_projects
]
fw_public_access_df = run_stackql_queries(fw_public_access_queries)

# forwarding rules
forwarding_rule_queries = [
    f"""
    select * from google.compute.forwarding_rules where project = '{project}'
    """
    for project in networked_projects
]
forwarding_rule_df = run_stackql_queries(forwarding_rule_queries)

## Network Summary

In [ ]:

# show network summary data
show_network_summary_cards(networks_df, len(interconnects_df), len(vpns_df), len(external_vpn_gateways_df), len(firewalls_df), len(fw_public_access_df), len(forwarding_rule_df))

## Network Peerings

In [ ]:

# get network peerings
network_peering_queries = [
    f"""
    SELECT 
    JSON_EXTRACT(peerings.value, '$.name') as peering_name,
    name,
    SPLIT_PART(JSON_EXTRACT(peerings.value, '$.network'), '/', 10) as peering_network,
    '{project}' as project,
    IIF((JSON_EXTRACT(peerings.value, '$.autoCreateRoutes') = 1), 'true', 'false') as autoCreateRoutes,
    IIF((JSON_EXTRACT(peerings.value, '$.exchangeSubnetRoutes') = 1), 'true', 'false') as exchangeSubnetRoutes,
    IIF((JSON_EXTRACT(peerings.value, '$.importCustomRoutes') = 1), 'true', 'false') as importCustomRoutes,
    IIF((JSON_EXTRACT(peerings.value, '$.exportCustomRoutes') = 1), 'true', 'false') as exportCustomRoutes,
    IIF((JSON_EXTRACT(peerings.value, '$.importSubnetRoutesWithPublicIp') = 1), 'true', 'false') as importSubnetRoutesWithPublicIp,
    IIF((JSON_EXTRACT(peerings.value, '$.exportSubnetRoutesWithPublicIp') = 1), 'true', 'false') as exportSubnetRoutesWithPublicIp,
    JSON_EXTRACT(peerings.value, '$.stackType') as stackType,
    JSON_EXTRACT(peerings.value, '$.state') as state,
    JSON_EXTRACT(peerings.value, '$.stateDetails') as stateDetails
    FROM google.compute.networks, json_each(peerings) peerings
    WHERE project = '{project}'
    AND peerings IS NOT NULL
    """
    for project in networked_projects
]
network_peerings_df = run_stackql_queries(network_peering_queries)
network_peerings_df

### Network Peering Graph

In [ ]:

# graph peerings
graph_network_peerings(network_peerings_df)

### Network Peering Details

In [ ]:

# show network peering details
network_peerings_df

## Shared VPC

In [ ]:

# get xpn networks
queries = [
    f"""
   SELECT 
   name 
   FROM 
   google.compute.projects
   WHERE project =  '{project}' AND xpnProjectStatus = 'HOST'
    """
    for project in networked_projects
]
xpn_hosts_df = run_stackql_queries(queries)
xpn_hosts = xpn_hosts_df['name'].tolist()

queries = [
    f"""
   SELECT 
   id as serviceProject
   ,'{project}' as hostProject
   FROM 
   google.compute.xpn_resources
   WHERE project =  '{project}'
    """
    for project in xpn_hosts
]
xpn_resources_df = run_stackql_queries(queries)

In [ ]:

# show tree
tree = generate_xpn_tree(xpn_resources_df)
tree

## Subnetworks

In [ ]:

# get and show subnetworks
queries = [
    f"""
    SELECT 
    name,
    SPLIT_PART(region, '/', 9) as region,
    SPLIT_PART(network, '/', 10) as network,
    '{project}' as project,
    stackType,
    ipCidrRange,
    secondaryIpRanges,
    gatewayAddress,
    ipv6CidrRange,
    purpose,
    enableFlowLogs,
    privateIpGoogleAccess
    FROM google.compute.subnetworks WHERE project = '{project}'
    """
    for project in networked_projects
]

subnetworks_df = run_stackql_queries(queries)
subnetworks_df

### Subnet IAM Policies

In [ ]:

# get subnet iam policies
queries = [
    f"""
    SELECT 
    '{row['name']}' as subnet
    ,'{row['project']}' as project
    ,'{row['region']}'  as region
    , role
    , condition
    , SPLIT_PART(members.value, ':', 1) as member_type
    , SPLIT_PART(members.value, ':', 2) as member
    FROM google.compute.subnetworks_iam_policies, json_each(members) members
    WHERE project = '{row['project']}' AND region = '{row['region']}' AND resource = '{row['name']}'
    """
    for _, row in subnetworks_df.iterrows()
]
subnetworks_bindings_df = run_stackql_queries(queries)
subnetworks_bindings_df

## Addresses

In [ ]:

# get addresses
address_queries = [
    f"""
    SELECT 
    name,
    '{project}' as project, 
    SPLIT_PART(network, '/', 10) as network,
    SPLIT_PART(subnetwork, '/', 11) as subnetwork,
    addressType,
    address,
    ipVersion,
    ipv6EndpointType,
    purpose,
    labels,
    status,
    SPLIT_PART(JSON_EXTRACT(users, '$[0]'), '/', 11) as user,
    networkTier
    FROM google.compute.addresses WHERE project = '{project}'
    """
    for project in networked_projects
]
addresses_df = run_stackql_queries(address_queries)

global_address_queries = [
    f"""
    select 
    name,
    description,
    '{project}' as project,
    SPLIT_PART(network, '/', 10) as network,
    labels,
    networkTier,
    users,
    addressType,
    address,
    purpose,
    ipVersion,
    prefixLength,
    status,
    subnetwork,
    ipv6EndpointType
    from google.compute.global_addresses where project = '{project}' and name is not null
    """
    for project in networked_projects
]
global_address_df = run_stackql_queries(global_address_queries)

### Address Details

In [ ]:

# show addresses
addresses_df

### Global Address Details

In [ ]:

# show global addresses
global_address_df

## Hybrid Connectivity

In [ ]:

# vpn tunnels
vpn_tunnels_queries = [
    f"""
    select 
    name,
    description,
    '{project}' as project,
    SPLIT_PART(region, '/', 9) as region,
    status,
    detailedStatus,
    SPLIT_PART(router, '/', 11) as router,
    SPLIT_PART(vpnGateway, '/', 11) as vpnGateway,
    targetVpnGateway,
    vpnGatewayInterface,
    peerGcpGateway,
    ikeVersion,
    peerIp,
    peerExternalGatewayInterface,
    SPLIT_PART(peerExternalGateway, '/', 10) as peerExternalGateway,
    localTrafficSelector,
    remoteTrafficSelector,
    labels
    from google.compute.vpn_tunnels where project = '{project}'
    """
    for project in networked_projects
]
vpn_tunnels_df = run_stackql_queries(vpn_tunnels_queries)

### VPN Gateways

In [ ]:

# show vpn gateways
vpns_df

### External VPN Gateways

In [ ]:

# show external vpn gateways
external_vpn_gateways_df

### VPN Tunnels

In [ ]:

# show vpn tunnels
vpn_tunnels_df

## Firewalls

In [ ]:

# show firewalls
firewalls_df

### Firewalls with Public Access

In [ ]:

# show firewalls with public access
fw_public_access_df

## Routers and Routes

In [ ]:

# routers
routers_queries = [
    f"""
    select 
    name
    ,description
    ,SPLIT_PART(region, '/', 9) as region	
    ,SPLIT_PART(network, '/', 10) as network
    ,nats
    ,bgp
    ,bgpPeers
    ,JSON_ARRAY_LENGTH(interfaces) as num_interfaces
    ,interfaces
    ,encryptedInterconnectRouter
    ,md5AuthenticationKeys		 	
    from google.compute.routers where project = '{project}'
    """
    for project in networked_projects
]
routers_df = run_stackql_queries(routers_queries)

routes_queries = [
    f"""
    select 
    name
    ,description
    ,'{project}' as project
    ,SPLIT_PART(network, '/', 10) as network
    ,SPLIT_PART(nextHopInstance, '/', 11) as nextHopInstance
    ,tags
    ,nextHopIlb	
    ,destRange	
    ,nextHopHub	
    ,SPLIT_PART(nextHopGateway, '/', 10) as nextHopGateway
    ,routeStatus
    ,priority
    ,nextHopVpnTunnel
    ,routeType
    ,nextHopIp
    ,warnings
    ,creationTimestamp
    ,asPaths
    ,nextHopPeering
    ,SPLIT_PART(nextHopNetwork, '/', 10) as nextHopNetwork
    from google.compute.routes where project = '{project}'
    """
    for project in networked_projects
]
routes_df = run_stackql_queries(routes_queries)

### Routers

In [ ]:

# show routers
routers_df

### Routes

In [ ]:

# show routes
routes_df